# II Autograd System
## Tensors and Gradients

In [ ]:
import torch

In [ ]:
# Create a tensor with gradient tracking
x = torch.tensor([1.0, 2.0, 3.0], requires_grad=True)
print(f'{x=}')

## Computing Gradients

In [ ]:
# Perform some operations
y = x * x  # y = x^2
z = y.mean()

# Compute gradients
z.backward()

# Access the gradient of x
print(f'{x=}, {x.grad=}')  # dz/dx = dy/dx evaluated at x = 2x

## Gradient Computation and Backpropagation

In [ ]:
x = torch.tensor([2.0, 3.0, 4.0], requires_grad=True)
y = x * x  # y = x^2
z = y.sum()  # z = sum(y)

# Compute gradients
z.backward()
print(f'{z=}, {z.grad=}\n{y=}, {y.grad=}\n{x=}, {x.grad=}\n')  # Outputs the gradient of z with respect to x = 2x * 1, UserWarning

# Calling backward again will accumulate the gradients
y = x * x
y.sum().backward()
print(f'{z=}, {z.grad=}\n{y=}, {y.grad=}\n{x=}, {x.grad=}\n')  # The gradients are accumulated, UserWarning

# Reset
x.grad.zero_()
# z, y.grad.zero() - leafes only
print(f'{z=}, {z.grad=}\n{y=}, {y.grad=}\n{x=}, {x.grad=}')  # UserWarning

## Chain Rule in Autograd

In [ ]:
x = torch.tensor([1.0, 2.0, 3.0], requires_grad=True)
y = x * x  # y = x^2
z = y.sum()  # z = sum(y)

z.backward()  # Compute gradients

# For each element in x, the gradient will be 2*x, since y = x^2 and z = sum(y) = 2x * 1
print(x.grad)  # Should print tensor([2.0, 4.0, 6.0])

## Detaching Tensors

In [ ]:
x = torch.tensor([1.0, 2.0, 3.0], requires_grad=True)
y = x * x

# Detach y from the computation graph
y_detached = y.detach()

# y_detached does not require gradients
print(y_detached.grad)

with torch.no_grad():
    z = x * 2  # Inside this block, no operations will track gradients
    print(z.requires_grad)  # Temporary False

In [ ]:
x = torch.tensor([1.0, 2.0, 3.0], requires_grad=True)
y = x * x
z = y + y
q = z.sum()

# y.backward()  # Error, grad can be implicitly created only for scalar outputs
q.backward()  # 2x * 1 + 2x * 1

print(f'{q=}, {q.grad=}\n{z=}, {z.grad=}\n{y=}, {y.grad=}\n{x=}, {x.grad=}')

In [ ]:
x = torch.tensor([1.0, 2.0, 3.0], requires_grad=True)
y = x * x
y1 = x * x
y1 = y1.detach()
z = y + y1
q = z.sum()

q.backward()  # 2x * 1 + (2x * 1 - detached)

print(f'{q=}, {q.grad=}\n{z=}, {z.grad=}\n{y=}, {y.grad=}\n{x=}, {x.grad=}')

## Custom Autograd Functions

In [ ]:
class MyReLU(torch.autograd.Function):
    """A custom operation with manually defined forward and backward passes."""
    @staticmethod
    def forward(ctx, input):
        ctx.save_for_backward(input)
        return input.clamp(min=0)

    @staticmethod
    def backward(ctx, grad_output):
        inpt, = ctx.saved_tensors
        grad_input = grad_output.clone()
        grad_input[inpt < 0] = 0
        return grad_input

x = torch.tensor([-1.0, 1.0, 2.0], requires_grad=True)
relu = MyReLU.apply
y = relu(x)
y.backward(torch.ones_like(x))
print(x.grad)